<a href="https://colab.research.google.com/github/Ph1lipXu/Biotechnological-Ecosystem-Service-Replacement-Research/blob/main/Modeling_16000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
#!pip install tensorflow nltk scikit-learn
#!pip install gensim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, SimpleRNN, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import gensim
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision = 4, suppress = True)

In [3]:
!wget https://raw.githubusercontent.com/Ph1lipXu/Machine-Learning-on-Suicide-and-Depression-Detection/refs/heads/main/data/cleaned_data_16000.csv

--2025-04-10 19:50:01--  https://raw.githubusercontent.com/Ph1lipXu/Machine-Learning-on-Suicide-and-Depression-Detection/refs/heads/main/data/cleaned_data_16000.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36085156 (34M) [text/plain]
Saving to: ‘cleaned_data_16000.csv’

cleaned_data_16000. 100%[===================>]  34.41M   112MB/s    in 0.3s    

2025-04-10 19:50:03 (112 MB/s) - ‘cleaned_data_16000.csv’ saved [36085156/36085156]



In [23]:
df = pd.read_csv("/content/cleaned_data_16000.csv")
df.head()

,text,class,tokens
0,can i get some support please...so i am not as depressed as i used to be (i ...,depression,"['can', 'I', 'get', 'some', 'support', 'please', 'so', 'I', 'be', 'not', 'as..."
1,"everything is going wrong .i have been trying not to drink, but everyone is ...",depression,"['everything', 'be', 'go', 'wrong', 'have', 'be', 'try', 'not', 'to', 'drink..."
2,i am done fighting it.*gone*,depression,"['I', 'be', 'do', 'fight', 'it', 'go']"
3,today i cut my hairmy hair has always been a thick mess of curls that went a...,depression,"['today', 'I', 'cut', 'my', 'hairmy', 'hair', 'have', 'always', 'be', 'a', '..."
4,i do not know what to do and i have no hopes for the future.it is kinda toug...,depression,"['I', 'do', 'not', 'know', 'what', 'to', 'do', 'and', 'I', 'have', 'no', 'ho..."


# Modeling

In [24]:
df['suicide_class'] = df['class'].apply(lambda x: 'suicide' if x == 'SuicideWatch' else 'nonsuicide')
df['depression_class'] = df['class'].apply(lambda x: 'depression' if x == 'depression' else 'nondepression')
df['teenager_class'] = df['class'].apply(lambda x: 'teenager' if x == 'teenagers' else 'nonteenager')

In [25]:
df.head(10)

,text,class,tokens,suicide_class,depression_class,teenager_class
0,can i get some support please...so i am not as depressed as i used to be (i ...,depression,"['can', 'I', 'get', 'some', 'support', 'please', 'so', 'I', 'be', 'not', 'as...",nonsuicide,depression,nonteenager
1,"everything is going wrong .i have been trying not to drink, but everyone is ...",depression,"['everything', 'be', 'go', 'wrong', 'have', 'be', 'try', 'not', 'to', 'drink...",nonsuicide,depression,nonteenager
2,i am done fighting it.*gone*,depression,"['I', 'be', 'do', 'fight', 'it', 'go']",nonsuicide,depression,nonteenager
3,today i cut my hairmy hair has always been a thick mess of curls that went a...,depression,"['today', 'I', 'cut', 'my', 'hairmy', 'hair', 'have', 'always', 'be', 'a', '...",nonsuicide,depression,nonteenager
4,i do not know what to do and i have no hopes for the future.it is kinda toug...,depression,"['I', 'do', 'not', 'know', 'what', 'to', 'do', 'and', 'I', 'have', 'no', 'ho...",nonsuicide,depression,nonteenager
5,"tired of life, tired of living. do not know what to do hey guys, \n\ni am 16...",depression,"['tired', 'of', 'life', 'tired', 'of', 'live', 'do', 'not', 'know', 'what', ...",nonsuicide,depression,nonteenager
6,what is one concrete thing that has helped you in your battle against depres...,depression,"['what', 'be', 'one', 'concrete', 'thing', 'that', 'have', 'help', 'you', 'i...",nonsuicide,depression,nonteenager
7,does mental health go hand in hand with the physical health?when i feel at m...,depression,"['do', 'mental', 'health', 'go', 'hand', 'in', 'hand', 'with', 'the', 'physi...",nonsuicide,depression,nonteenager
8,the thing that hurts the most is knowing that i have been through worse.when...,depression,"['the', 'thing', 'that', 'hurt', 'the', 'most', 'be', 'know', 'that', 'I', '...",nonsuicide,depression,nonteenager
9,need someone to talk toi am a guy in high school and i just need to talk to ...,depression,"['need', 'someone', 'to', 'talk', 'toi', 'be', 'a', 'guy', 'in', 'high', 'sc...",nonsuicide,depression,nonteenager


In [61]:
def build_cnn_model(vocab_size, embedding_matrix, num_classes):
    model = Sequential([
        Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=True),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_rnn_model(vocab_size, embedding_matrix, num_classes):
    model = Sequential([
        Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=True),
        LSTM(128, return_sequences=False),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_bilstm_model(vocab_size, embedding_matrix, num_classes):
    model = Sequential([
        Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=True),
        Bidirectional(LSTM(128)),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_and_evaluate(model, train_padded, train_labels, test_padded, test_labels,
                       class_weights=None, epochs=10, batch_size=32):
    f1_callback = F1ScoreCallback(validation_data=(test_padded, test_labels))

    model.fit(train_padded, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(test_padded, test_labels),
              callbacks=[f1_callback],
              class_weight=class_weights)

    loss, acc = model.evaluate(test_padded, test_labels)
    print(f"Test Accuracy: {acc:.4f}")

## Suicide / Non-Suicide

In [40]:
# Encode labels
label_encoder = LabelEncoder()
df["suicide_label"] = label_encoder.fit_transform(df["suicide_class"])
num_classes = len(label_encoder.classes_)

In [41]:
from sklearn.model_selection import train_test_split

# Instead of random train test split, stratify by class
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["tokens"], df["suicide_label"], random_state=64, stratify=df['suicide_label']
)
print('Training data: ',len(train_texts))
print('Testing data: ',len(test_texts))

Training data:  12000
Testing data:  4000


In [42]:
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [43]:
print(train_texts.head(10))

1435     ['anyone', 'else', 'feel', 'like', 'this', 'I', 'have', 'be', 'cope', 'with'...
1368     ['lose', 'my', 'sense', 'of', 'realityit', 'be', 'another', 'one', 'of', 'th...
5863                                          ['good', 'way', 'to', 'commit', 'suicide']
8929     ['I', 'be', 'bakk', 'you', 'lousy', 'son', 'of', 'bitch', 'thoughught', 'I',...
15448    ['firstly', 'what', 'be', 'your', 'thoughught', 'on', 'the', 'titular', 'cha...
11663    ['just', 'realize', 'how', 'close', 'my', 'teenage', 'year', 'be', 'to', 'fi...
3897     ['just', 'a', 'thoughught', 'that', 'enter', 'my', 'mind', 'lie', 'in', 'bed...
4635     ['why', 'when', 'I', 'die', 'the', 'world', 'will', 'not', 'stop', 'spin', '...
14158    ['we', 'just', 'have', 'our', 'concrete', 'foundation', 'reinforce', 'so', '...
13454    ['my', 'wife', 'be', 'italian', 'and', 'my', 'do', 'not', 'speak', 'very', '...
Name: tokens, dtype: object


### Vectorization/Embedding

In [45]:
# Initialize tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding sequences to have the same length
max_len = 200  # Max length for padding
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Vocabulary size
vocab_size = len(word_index) + 1

In [46]:
word2vec_model = gensim.models.Word2Vec(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)
fasttext_model = gensim.models.FastText(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)

embedding_matrix_w2v = np.zeros((vocab_size, 100))
embedding_matrix_ft = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix_w2v[i] = word2vec_model.wv[word]
    if word in fasttext_model.wv:
        embedding_matrix_ft[i] = fasttext_model.wv[word]

In [47]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score
class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_data, val_labels = self.validation_data
        val_preds = self.model.predict(val_data)
        val_preds = np.argmax(val_preds, axis=1)  # Convert probabilities to class labels
        val_labels = np.argmax(val_labels, axis=1)  # Convert one-hot labels to class labels

        f1 = f1_score(val_labels, val_preds, average='weighted')
        print(f" - val_f1: {f1:.4f}")
        logs["val_f1"] = f1

In [62]:
from sklearn.utils import class_weight

# Compute class weights
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df['suicide_label']),
    y=df['suicide_label']
)
class_weights = dict(enumerate(weights))

In [63]:
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels,
                   class_weights=class_weights)

Training Bi-LSTM with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
 - val_f1: 0.6353
375/375 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.6674 - loss: 0.5872 - val_accuracy: 0.6155 - val_loss: 0.5263 - val_f1: 0.6353
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.7988
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.7411 - loss: 0.4736 - val_accuracy: 0.7865 - val_loss: 0.4182 - val_f1: 0.7988
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8027
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.8405 - loss: 0.3327 - val_accuracy: 0.7905 - val_loss: 0.4634 - val_f1: 0.8027
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
 - val_f1: 0.8163
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.8850 - loss: 0.2600 - val_accuracy: 0.8067 - val_loss: 0.4375 - val_f1: 0.8163
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
 - val_f1: 0.8257
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.9059 - loss: 0.2121 - val_accuracy: 0.8188 - val_loss: 0.4303 

In [64]:
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.7970
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.7742 - loss: 0.5068 - val_accuracy: 0.8087 - val_loss: 0.4039 - val_f1: 0.7970
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
 - val_f1: 0.8007
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.8523 - loss: 0.3402 - val_accuracy: 0.8198 - val_loss: 0.3941 - val_f1: 0.8007
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8211
375/375 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - accuracy: 0.8898 - loss: 0.2538 - val_accuracy: 0.8198 - val_loss: 0.4072 - val_f1: 0.8211
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8190
375/375 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - accuracy: 0.9264 - loss: 0.1867 - val_accuracy: 0.8195 - val_loss: 0.4418 - val_f1: 0.8190
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8221
375/375 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.9489 - loss: 0.1337 - val_accuracy: 0.8227 - val_loss: 0.53

In [65]:
print("Training Bi-LSTM with FastText Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8035
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.7773 - loss: 0.4853 - val_accuracy: 0.8198 - val_loss: 0.4000 - val_f1: 0.8035
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
 - val_f1: 0.8355
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.8660 - loss: 0.3100 - val_accuracy: 0.8370 - val_loss: 0.3723 - val_f1: 0.8355
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8296
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.8970 - loss: 0.2428 - val_accuracy: 0.8255 - val_loss: 0.4211 - val_f1: 0.8296
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8196
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.9151 - loss: 0.2090 - val_accuracy: 0.8185 - val_loss: 0.4451 - val_f1: 0.8196
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
 - val_f1: 0.8149
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.9406 - loss: 0.1542 - val_accuracy: 0.8205 - val_loss: 0.5147 -

In [66]:
print("Training CNN with Word2Vec Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8331
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7714 - loss: 0.5331 - val_accuracy: 0.8385 - val_loss: 0.3439 - val_f1: 0.8331
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8485
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8688 - loss: 0.2904 - val_accuracy: 0.8487 - val_loss: 0.3373 - val_f1: 0.8485
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8425
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9320 - loss: 0.1840 - val_accuracy: 0.8413 - val_loss: 0.3667 - val_f1: 0.8425
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
 - val_f1: 0.8186
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9670 - loss: 0.0978 - val_accuracy: 0.8270 - val_loss: 0.5663 - val_f1: 0.8186
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8230
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9893 - loss: 0.0385 - val_accuracy: 0.8282 - val_loss: 0.6852 - val_f1

In [67]:
print("Training CNN with FastText Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8440
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7755 - loss: 0.4949 - val_accuracy: 0.8445 - val_loss: 0.3369 - val_f1: 0.8440
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8407
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8833 - loss: 0.2743 - val_accuracy: 0.8445 - val_loss: 0.3344 - val_f1: 0.8407
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8422
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9321 - loss: 0.1786 - val_accuracy: 0.8440 - val_loss: 0.3954 - val_f1: 0.8422
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8334
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9731 - loss: 0.0870 - val_accuracy: 0.8363 - val_loss: 0.5009 - val_f1: 0.8334
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8384
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9906 - loss: 0.0353 - val_accuracy: 0.8375 - val_loss: 0.5947 - val_f1

In [68]:
print("Training RNN with Word2Vec Embeddings...")
rnn_model = build_rnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(rnn_model, train_padded, train_labels, test_padded, test_labels)

Training RNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - val_f1: 0.7167
375/375 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.7514 - loss: 0.5789 - val_accuracy: 0.7650 - val_loss: 0.5300 - val_f1: 0.7167
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7118
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.7511 - loss: 0.5436 - val_accuracy: 0.7538 - val_loss: 0.5349 - val_f1: 0.7118
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
 - val_f1: 0.7010
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.7932 - loss: 0.4921 - val_accuracy: 0.7535 - val_loss: 0.5326 - val_f1: 0.7010
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7580
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8168 - loss: 0.4601 - val_accuracy: 0.7878 - val_loss: 0.5466 - val_f1: 0.7580
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7190
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.8187 - loss: 0.4284 - val_accuracy: 0.7607 - val_loss: 0.5666 - 

In [69]:
print("Training RNN with FastText Embeddings...")
rnn_model = build_rnn_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(rnn_model, train_padded, train_labels, test_padded, test_labels)

Training RNN with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - val_f1: 0.6920
375/375 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.7461 - loss: 0.5696 - val_accuracy: 0.7560 - val_loss: 0.5407 - val_f1: 0.6920
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7306
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7780 - loss: 0.5223 - val_accuracy: 0.7700 - val_loss: 0.5190 - val_f1: 0.7306
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
 - val_f1: 0.7134
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.7905 - loss: 0.4934 - val_accuracy: 0.7605 - val_loss: 0.5480 - val_f1: 0.7134
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7131
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8145 - loss: 0.4524 - val_accuracy: 0.7570 - val_loss: 0.5686 - val_f1: 0.7131
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7185
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8231 - loss: 0.4342 - val_accuracy: 0.7575 - val_loss: 0.5690 - v

## Depression / Non-Depression

In [70]:
# Encode labels
label_encoder = LabelEncoder()
df["depression_label"] = label_encoder.fit_transform(df["depression_class"])
num_classes = len(label_encoder.classes_)

In [71]:
from sklearn.model_selection import train_test_split

# Instead of random train test split, stratify by class
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["tokens"], df["depression_label"], random_state=64, stratify=df['depression_label']
)

In [72]:
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

### Vectorization/Embedding

In [73]:
# Initialize tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding sequences to have the same length
max_len = 200  # Max length for padding
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Vocabulary size
vocab_size = len(word_index) + 1

In [74]:
word2vec_model = gensim.models.Word2Vec(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)
fasttext_model = gensim.models.FastText(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)

embedding_matrix_w2v = np.zeros((vocab_size, 100))
embedding_matrix_ft = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix_w2v[i] = word2vec_model.wv[word]
    if word in fasttext_model.wv:
        embedding_matrix_ft[i] = fasttext_model.wv[word]

In [75]:
from sklearn.utils import class_weight

# Compute class weights
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df['depression_label']),
    y=df['depression_label']
)
class_weights = dict(enumerate(weights))

In [76]:
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
 - val_f1: 0.6983
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - accuracy: 0.7485 - loss: 0.5521 - val_accuracy: 0.7663 - val_loss: 0.4287 - val_f1: 0.6983
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.7454
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8125 - loss: 0.3875 - val_accuracy: 0.7832 - val_loss: 0.4035 - val_f1: 0.7454
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
 - val_f1: 0.7951
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.8617 - loss: 0.3114 - val_accuracy: 0.7903 - val_loss: 0.4207 - val_f1: 0.7951
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.7811
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.8898 - loss: 0.2411 - val_accuracy: 0.7995 - val_loss: 0.5301 - val_f1: 0.7811
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8099
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9341 - loss: 0.1594 - val_accuracy: 0.8095 - val_loss: 0.4854 

In [77]:
print("Training Bi-LSTM with FastText Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
 - val_f1: 0.8157
375/375 ━━━━━━━━━━━━━━━━━━━━ 13s 28ms/step - accuracy: 0.7584 - loss: 0.4828 - val_accuracy: 0.8123 - val_loss: 0.3996 - val_f1: 0.8157
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8259
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.8387 - loss: 0.3430 - val_accuracy: 0.8260 - val_loss: 0.4083 - val_f1: 0.8259
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8140
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.8901 - loss: 0.2453 - val_accuracy: 0.8095 - val_loss: 0.4326 - val_f1: 0.8140
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8101
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9224 - loss: 0.1776 - val_accuracy: 0.8150 - val_loss: 0.4886 - val_f1: 0.8101
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8014
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.9425 - loss: 0.1404 - val_accuracy: 0.8048 - val_loss: 0.5474 

In [78]:
print("Training CNN with Word2Vec Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8118
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.7640 - loss: 0.5119 - val_accuracy: 0.8250 - val_loss: 0.3731 - val_f1: 0.8118
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8317
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8557 - loss: 0.3079 - val_accuracy: 0.8313 - val_loss: 0.3577 - val_f1: 0.8317
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.7970
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9273 - loss: 0.1817 - val_accuracy: 0.8177 - val_loss: 0.5090 - val_f1: 0.7970
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8122
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9737 - loss: 0.0781 - val_accuracy: 0.8185 - val_loss: 0.5397 - val_f1: 0.8122
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8018
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9927 - loss: 0.0265 - val_accuracy: 0.8145 - val_loss: 0.8568 - val_f1

In [79]:
print("Training CNN with FastText Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8059
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7704 - loss: 0.5051 - val_accuracy: 0.8192 - val_loss: 0.3744 - val_f1: 0.8059
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8097
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8501 - loss: 0.3223 - val_accuracy: 0.8265 - val_loss: 0.4098 - val_f1: 0.8097
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8290
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9141 - loss: 0.2101 - val_accuracy: 0.8250 - val_loss: 0.4030 - val_f1: 0.8290
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8162
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9683 - loss: 0.0967 - val_accuracy: 0.8188 - val_loss: 0.5211 - val_f1: 0.8162
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8224
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9921 - loss: 0.0305 - val_accuracy: 0.8242 - val_loss: 0.6704 - val_f1

In [80]:
print("Training RNN with Word2Vec Embeddings...")
rnn_model = build_rnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(rnn_model, train_padded, train_labels, test_padded, test_labels)

Training RNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - val_f1: 0.7081
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.7384 - loss: 0.6063 - val_accuracy: 0.7670 - val_loss: 0.5211 - val_f1: 0.7081
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7498
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7836 - loss: 0.5123 - val_accuracy: 0.7772 - val_loss: 0.5116 - val_f1: 0.7498
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7355
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.8023 - loss: 0.4893 - val_accuracy: 0.7763 - val_loss: 0.5201 - val_f1: 0.7355
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7472
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8225 - loss: 0.4540 - val_accuracy: 0.7812 - val_loss: 0.5400 - val_f1: 0.7472
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
 - val_f1: 0.7442
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8346 - loss: 0.4286 - val_accuracy: 0.7805 - val_loss: 0.5503 - v

In [81]:
print("Training RNN with FastText Embeddings...")
rnn_model = build_rnn_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(rnn_model, train_padded, train_labels, test_padded, test_labels)

Training RNN with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - val_f1: 0.7230
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.7401 - loss: 0.5572 - val_accuracy: 0.7720 - val_loss: 0.5170 - val_f1: 0.7230
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7443
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.7786 - loss: 0.5214 - val_accuracy: 0.7775 - val_loss: 0.5122 - val_f1: 0.7443
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
 - val_f1: 0.7359
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.8144 - loss: 0.4624 - val_accuracy: 0.7778 - val_loss: 0.5287 - val_f1: 0.7359
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - val_f1: 0.7088
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.8253 - loss: 0.4464 - val_accuracy: 0.7703 - val_loss: 0.5893 - val_f1: 0.7088
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.7762
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8388 - loss: 0.4153 - val_accuracy: 0.7933 - val_loss: 0.5228 -

## Teenager / Non-Teenager


In [82]:
# Encode labels
label_encoder = LabelEncoder()
df["teenager_label"] = label_encoder.fit_transform(df["teenager_class"])
num_classes = len(label_encoder.classes_)

In [83]:
from sklearn.model_selection import train_test_split

# Instead of random train test split, stratify by class
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["tokens"], df["teenager_label"], random_state=64, stratify=df['teenager_label']
)

In [84]:
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

### Vectorization/Embedding

In [85]:
# Initialize tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding sequences to have the same length
max_len = 200  # Max length for padding
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Vocabulary size
vocab_size = len(word_index) + 1

In [86]:
word2vec_model = gensim.models.Word2Vec(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)
fasttext_model = gensim.models.FastText(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)

embedding_matrix_w2v = np.zeros((vocab_size, 100))
embedding_matrix_ft = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix_w2v[i] = word2vec_model.wv[word]
    if word in fasttext_model.wv:
        embedding_matrix_ft[i] = fasttext_model.wv[word]

In [87]:
from sklearn.utils import class_weight

# Compute class weights
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df['teenager_label']),
    y=df['teenager_label']
)
class_weights = dict(enumerate(weights))

In [88]:
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8771
375/375 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.8016 - loss: 0.4572 - val_accuracy: 0.8863 - val_loss: 0.2821 - val_f1: 0.8771
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
 - val_f1: 0.9126
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.9353 - loss: 0.1992 - val_accuracy: 0.9147 - val_loss: 0.2329 - val_f1: 0.9126
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.9027
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.9687 - loss: 0.1057 - val_accuracy: 0.9070 - val_loss: 0.2623 - val_f1: 0.9027
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
 - val_f1: 0.9070
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.9770 - loss: 0.0701 - val_accuracy: 0.9087 - val_loss: 0.2910 - val_f1: 0.9070
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.9109
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.9840 - loss: 0.0507 - val_accuracy: 0.9107 - val_loss: 0.3357 -

In [89]:
print("Training Bi-LSTM with FastText Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.8916
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - accuracy: 0.8180 - loss: 0.4250 - val_accuracy: 0.8938 - val_loss: 0.2627 - val_f1: 0.8916
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.9100
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.9280 - loss: 0.2056 - val_accuracy: 0.9100 - val_loss: 0.2329 - val_f1: 0.9100
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.9058
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.9620 - loss: 0.1123 - val_accuracy: 0.9047 - val_loss: 0.2673 - val_f1: 0.9058
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
 - val_f1: 0.9140
375/375 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9791 - loss: 0.0732 - val_accuracy: 0.9158 - val_loss: 0.3019 - val_f1: 0.9140
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
 - val_f1: 0.9116
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9843 - loss: 0.0502 - val_accuracy: 0.9133 - val_loss: 0.3035 

In [90]:
print("Training CNN with Word2Vec Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8974
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7617 - loss: 0.5822 - val_accuracy: 0.8985 - val_loss: 0.2596 - val_f1: 0.8974
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.9130
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9259 - loss: 0.2030 - val_accuracy: 0.9128 - val_loss: 0.2225 - val_f1: 0.9130
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
 - val_f1: 0.9151
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9754 - loss: 0.0832 - val_accuracy: 0.9160 - val_loss: 0.2381 - val_f1: 0.9151
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.9123
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9916 - loss: 0.0303 - val_accuracy: 0.9130 - val_loss: 0.2895 - val_f1: 0.9123
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.9040
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9984 - loss: 0.0073 - val_accuracy: 0.9032 - val_loss: 0.3851 - val_f1

In [91]:
print("Training CNN with FastText Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.8911
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.7738 - loss: 0.4820 - val_accuracy: 0.8978 - val_loss: 0.2542 - val_f1: 0.8911
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.9203
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9420 - loss: 0.1661 - val_accuracy: 0.9218 - val_loss: 0.2047 - val_f1: 0.9203
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.9154
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9822 - loss: 0.0625 - val_accuracy: 0.9150 - val_loss: 0.2468 - val_f1: 0.9154
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.9135
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9933 - loss: 0.0246 - val_accuracy: 0.9162 - val_loss: 0.3262 - val_f1: 0.9135
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
 - val_f1: 0.9117
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9975 - loss: 0.0088 - val_accuracy: 0.9122 - val_loss: 0.3670 - val_f1

In [92]:
print("Training RNN with Word2Vec Embeddings...")
rnn_model = build_rnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(rnn_model, train_padded, train_labels, test_padded, test_labels)

Training RNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - val_f1: 0.6429
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.7426 - loss: 0.5432 - val_accuracy: 0.7500 - val_loss: 0.5160 - val_f1: 0.6429
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - val_f1: 0.6429
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.7469 - loss: 0.5104 - val_accuracy: 0.7500 - val_loss: 0.5145 - val_f1: 0.6429
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.6429
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.7489 - loss: 0.4920 - val_accuracy: 0.7500 - val_loss: 0.5713 - val_f1: 0.6429
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.8387
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.7732 - loss: 0.4102 - val_accuracy: 0.8390 - val_loss: 0.3763 - val_f1: 0.8387
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
 - val_f1: 0.8632
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.8582 - loss: 0.3274 - val_accuracy: 0.8643 - val_loss: 0.3678 

In [93]:
print("Training RNN with FastText Embeddings...")
rnn_model = build_rnn_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(rnn_model, train_padded, train_labels, test_padded, test_labels)

Training RNN with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - val_f1: 0.6429
375/375 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.7457 - loss: 0.5562 - val_accuracy: 0.7500 - val_loss: 0.5060 - val_f1: 0.6429
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.6429
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.7468 - loss: 0.5090 - val_accuracy: 0.7500 - val_loss: 0.5094 - val_f1: 0.6429
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.6429
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.7568 - loss: 0.4820 - val_accuracy: 0.7500 - val_loss: 0.5199 - val_f1: 0.6429
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.8376
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7803 - loss: 0.3936 - val_accuracy: 0.8313 - val_loss: 0.3781 - val_f1: 0.8376
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
 - val_f1: 0.8623
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.8765 - loss: 0.2885 - val_accuracy: 0.8620 - val_loss: 0.3564 -